# Anwendung an einem torsionsweichen Trägerrost

Das vorangegangene Kapitel zeigt die Anwendung der Modellbildung an einem zweidimensionalen Stabtragwerk. Mit dem Anwendungsbeispiel des torsionsweichen Trägerrosts wird das Ziel verfolgt, ein dreidimensionales System mit dem Stabsystem abzubilden. 

## Aufgabenbeschrieb

Das folgende Beispiel stammt aus @marti_baustatik_2014. In der @fig-trm_uebersicht ist der Grundriss des Trägerrosts dargestellt. Es handelt sich um insgesamt 16 torsionsweiche Stäbe, welche an den Enden eingespannt sind.

![Grundriss des torsionsweichen Trägerrosts, neugezeichnet nach @marti_baustatik_2014](../imgs/trm_uebersicht.svg){#fig-trm_uebersicht}

Im Beispiel wird eine analytische Lösung zur Traglast aufgezeigt. Das Ziel ist es, mittels dem nicht-linearen FE-Modell eine numerische Lösung des unteren Grenzwerts der Traglast zu erhalten.

## Analytische Lösung

Die Analytische Lösung basiert auf dem Traglastverfahren. Mittels einem zulässigen Mechanismus wird ein oberer Grenzwert der Traglast hergeleitet. In der @fig-trm_schnitt sind zwei Stäbe dargestellt. Aus symmetriegründen lässt sich der obere Grenzwert des gesamten Trägerrosts anhand dieser Darstellung ermitteln.

![Schnitt des torsionsweichen Trägerrosts, mit dem angenommenen Mechanismus für den Innen- und Aussenträger](../imgs/trm_schnitt.svg){#fig-trm_schnitt}



Die äussere Arbeit $W_a$ des dargestellten Systems in @fig-trm_schnitt beträgt dabei für die am Rand gelegenen Stäbe (Punkte 2'45):

$$
W_{a,2'45} = 4 \cdot \left( Q \cdot \frac{1}{3} + Q \cdot \frac{1}{9} \right)
$$

Und für die Innenträger:

$$
W_{a,123} = 4 \cdot \left( Q \cdot 1 + Q \cdot \frac{1}{3} \right)
$$


Sowie beträgt die innere Arbeit:

$$
W_i = 8 \cdot \left(M_u' + M_u\right) \cdot \left(\frac{1}{3l} +\frac{1}{9l}\right)
$$

Durch das abschliessende Gleichsetzen der Arbeiten und das Lösen nach $Q$ folgt der obere Grenzwert der Traglast zu:

$$
Q = \frac{M_u + M_u'}{2 \cdot l}
$$

Die Plastizitätskontrolle ist in der @fig-trm_schnitt_my und @fig-trm_schnitt_vz gezeigt. Dabei wird eine Lastverteilung von je einer Hälfte $Q$ in $x$ und $y$ Richtung angenommen. Welche sich nach vollständigen Umlagern der Biegemomente einstellt.

![Plastizitätskontrolle anhand der Schnittgrössen der Biegemomente](../imgs/trm_schnitt_my.svg){#fig-trm_schnitt_my}

![Plastizitätskontrolle anhand der Schnittgrössen der Querkräfte](../imgs/trm_schnitt_vz.svg){#fig-trm_schnitt_vz}

Aus der Plastizitätskontrolle geht heraus, dass der Biegewiderstand nirgends überschritten wird, somit deckt sich der untere und obere Grenzwert der Traglast, sprich die Traglast $Q_u$ wurde gefunden.


$$
Q_u = \frac{M_u + M_u'}{2 \cdot l}
$$



## NLFE-Modell

Abschliessend wird die analytische Lösung mit der numerischen Lösung verglichen. Dabei wird das nicht-lineare FE-Modell erstellt.
Zunächst sind die Variablen mit numerischen Werten zu substituieren:

In [1]:
# | echo: false

from packages import *

In [2]:
# Rastergrösse
l = 1. * un.m  

# Positiver Biegewiderstand
M_u = 100 * un.kN * un.m  

# Negativer Biegewiderstand
M_apos_u = 100 * un.kN * un.m  

put_out()

$$
\begin{aligned}
l& = 1.0 \ \mathrm{m} \quad & M_{u}& = 100 \ \mathrm{kN} \cdot \mathrm{m} \quad & {M}'_{u}& = 100 \ \mathrm{kN} \cdot \mathrm{m} \end{aligned}
$$

Die geforderte Traglast des Systems liegt gemäss der analytischen Lösung für die gewählten Werte bei:

In [3]:

# Traglast
Q_u = (M_u + M_apos_u) / (2*l)

put_out(symbolic=True, rows=1)

$$
\begin{aligned}
Q_{u}& = \frac{M_{u} + {M}'_{u}}{2 \cdot l} = 100.0 \ \mathrm{kN} \end{aligned}
$$

### Makromodellbildung

Der Trägerrost wurde mittels dem Federmodell nachmodelliert. Dabei sind biegestarre, jedoch torsionsweiche Stäbe mittels Stabendgelenken gekoppelt. Dargestellt in der @fig-trm_geom.

![Statisches Modell in AxisVM des Trägerrosts](../imgs/trm_system.svg){#fig-trm_geom}


### Mikromodellbildung

In [4]:
# | echo: false

radians = np.linspace(-0.2, 0.2, 100)
M = np.piecewise(radians, [radians < 0, radians > 0], [-100, 100])

fig, ax = plt.subplots()

ax.plot(radians, M, linestyle="-", color="k")
ax.plot(radians, radians * 0, color="gray")

# Set axis labels
ax.set_xlabel(r"$\varphi$ [rad]")
ax.set_ylabel(r"$M$ [kNm]")

# Set limits
# ax.set_xlim([-0.2, 0.2])
# ax.set_ylim([-101, 101])

# Show the plot
plt.tight_layout()
plt.savefig("imgs/trm_gelenkdef.svg")
plt.close()

![Ideal-plastisches Momenten-Verdrehungs-Verhalten der Gelenke](../imgs/trm_gelenkdef.svg){#fig-trm_gelenkdef}


Den Gelenken wurde die Defintion gemäss @fig-trm_gelenkdef hinterlegt. Dabei wurde der Biegewiderstand in positiver und negativer Dimension angesetzt, ab dem Erreichen des Biegewiderstands fliesst das Gelenk. Dies führt dazu, dass sich die Biegemomente umlagern, sprich der Biegewiderstand auch in Trägermitte erreicht werden kann.


### Resultate

Bei der Traglastermittlung des Systems ist das Rotationsvermögen des Systems von keiner bedeutung. Die Traglast stellt sich bei einer vollständigen Umlagerung sämtlicher Biegemomente ein. Die folgenden Abbildungen zeigen die Verteilung der Biegemomente und der Querkräfte im Trägerrost, ermittelt mit dem NLFE-Modell.


![Biegemomente des Trägerrosts bei einer Belastung von 58 un.kN, erste plastische Gelenke entstehen bei der Lagerung der Innenträger](../imgs/trm_my58.svg){#fig-trm_my58}

![Biegemomente des Trägerrosts bei einer Belastung von 88 un.kN, plastische Gelenke entstehen bei der Lagerung der Aussenträger](../imgs/trm_my88.svg){#fig-trm_my88}

![Biegemomente des Trägerrosts bei einer Belastung von 94 un.kN, plastische Gelenke entstehen im Feld der Innenträger](../imgs/trm_my94.svg){#fig-trm_my94}

![Biegemomente des Trägerrosts bei einer Belastung von 100 un.kN, plastische Gelenke entstehen im Feld der Aussenträger](../imgs/trm_my100.svg){#fig-trm_my100}


![Querkräfte des Trägerrosts bei einer Belastung von 58 un.kN, erste plastische Gelenke entstehen bei der Lagerung der Innenträger](../imgs/trm_vz58.svg){#fig-trm_vz58}

![Querkräfte des Trägerrosts bei einer Belastung von 88 un.kN, plastische Gelenke entstehen bei der Lagerung der Aussenträger](../imgs/trm_vz88.svg){#fig-trm_vz88}

![Querkräfte des Trägerrosts bei einer Belastung von 94 un.kN, plastische Gelenke entstehen im Feld der Innenträger](../imgs/trm_vz94.svg){#fig-trm_vz94}

![Querkräfte des Trägerrosts bei einer Belastung von 100 un.kN, plastische Gelenke entstehen im Feld der Aussenträger](../imgs/trm_vz100.svg){#fig-trm_vz100}

Mit der Belastung aus dem unteren Grenzwert stellen sich exakt die Extremwerte der Biegemomente $M_u$ und $M'_u$ in den innen liegenden Stäben ein. Das Federmodell liefert den exakten unteren Grenzwert der Traglast. Dargestell in der @fig-trm_my100.

## Fazit

-